<a href="https://colab.research.google.com/github/hiydavid/numerai-models/blob/main/numerai_sklearn_model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numerai SKLearn Model * WIP *
---

Base model
* RF with SKLearn

---
# Load data & libraries

In [1]:
# set drive location
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)
path = '/content/drive/MyDrive/Colab/numerai/saved_models/'

Mounted at /content/drive/


In [2]:
# install dependencies
!pip install -Uqq numerapi==2.4.5

In [12]:
# import dependencies
import pandas as pd
import numerapi
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import pickle

In [4]:
# download the latest training dataset (takes around 30s)
training_data_url = "https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz" 
training_data = pd.read_csv(training_data_url)

--- 
# Training

In [5]:
# split training into training & validation
training_data, testing_data = train_test_split(training_data, test_size = 0.2)

training_data.shape, testing_data.shape

((401446, 314), (100362, 314))

In [6]:
# find only the feature columns
feature_cols = training_data.columns[training_data.columns.str.startswith('feature')]

In [7]:
# select those columns out of the training dataset
training_features = training_data[feature_cols]
testing_features = testing_data[feature_cols]

In [8]:
# create a model and fit the training data (~30 sec to run)
model = RandomForestRegressor(n_estimators = 50, max_depth = 5)
model.fit(training_features, training_data.target)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [13]:
# evaluate
r2 = r2_score(testing_data.target, model.predict(testing_features))
mse = mean_squared_error(testing_data.target, model.predict(testing_features))
print('r2:', round(r2, 3)), print('mse:', round(mse, 3));

r2: 0.001
mse: 0.05


---
# Save model

In [14]:
# pickle the model
pkl_filename = "sklearn_rf_model.pkl"
with open(path + pkl_filename, 'wb') as file:
    pickle.dump(model, file)